In [1]:
# standard imports
import os

# extra imports
import numpy as np
import pandas as pd

from ..utils.helper_functions import load_config

In [2]:
# load config file
config = load_config()

## Join Data

In [3]:
# read in raw data
# add neighborhood indicator and KitchenAbvGr for CollegeCr dataset
df_college = pd.read_csv(os.path.join(config['path']['raw'], 'CollegeCr.csv'))
df_college['neighborhood'] = 'CollegeCr'
df_college['KitchenAbvGr'] = np.nan
df_college['uniqueID'] = 'train'

df_college_test = pd.read_csv(os.path.join(config['path']['raw'], 'CollegeCr.test.csv'))
df_college_test['neighborhood'] = 'CollegeCr'
df_college_test['KitchenAbvGr'] = np.nan

# add neighborhood indicator and BsmtUnfSF for Edwards dataset
df_edwards = pd.read_csv(os.path.join(config['path']['raw'], 'Edwards.csv'))
df_edwards['neighborhood'] = 'Edwards'
df_edwards['BsmtUnfSF'] = np.nan
df_edwards['uniqueID'] = 'train'

df_edwards_test = pd.read_csv(os.path.join(config['path']['raw'], 'Edwards.test.csv'))
df_edwards_test['neighborhood'] = 'Edwards'
df_edwards_test['BsmtUnfSF'] = np.nan

# add neighborhood indicator for OldTown dataset
df_oldtown = pd.read_csv(os.path.join(config['path']['raw'], 'OldTown.csv'))
df_oldtown['neighborhood'] = 'OldTown'
df_oldtown['uniqueID'] = 'train'

df_oldtown_test = pd.read_csv(os.path.join(config['path']['raw'], 'OldTown.test.csv'))
df_oldtown_test['neighborhood'] = 'OldTown'

# full data set
df_lst = [df_college, df_edwards, df_oldtown, df_college_test, df_edwards_test, df_oldtown_test]
df_full = pd.concat(df_lst)
df_full.head()

,OverallQual,BedroomAbvGr,CentralAir,BsmtQual,BsmtUnfSF,Fireplaces,YrSold,HouseStyle,HeatingQC,GarageType,...,YearBuilt,BsmtCond,Foundation,Electrical,BldgType,OverallCond,GrLivArea,neighborhood,KitchenAbvGr,uniqueID
0,7,3,Y,Gd,150.0,0,2008,2Story,Ex,Attchd,...,2003,TA,PConc,SBrkr,1Fam,5,1710,CollegeCr,NaN,train
1,7,3,Y,Gd,434.0,1,2008,2Story,Ex,Attchd,...,2001,TA,PConc,SBrkr,1Fam,5,1786,CollegeCr,NaN,train
2,8,3,Y,Gd,1777.0,1,2008,1Story,Ex,Attchd,...,2002,TA,PConc,SBrkr,1Fam,5,1795,CollegeCr,NaN,train
3,8,3,Y,Ex,1234.0,0,2008,1Story,Ex,Attchd,...,2007,TA,PConc,SBrkr,1Fam,5,1234,CollegeCr,NaN,train
4,5,3,Y,Gd,167.0,0,2008,1Story,TA,Detchd,...,1975,TA,CBlock,SBrkr,1Fam,6,938,CollegeCr,NaN,train


In [4]:
# remove from workspace
del df_college, df_edwards, df_oldtown, df_college_test, df_edwards_test, df_oldtown_test
del df_lst

## Cleaning

In [5]:
# split lot info and drop
df_full[['LotConfig', 'LotShape', 'LotArea', 'LotFrontage']] = df_full['LotInfo'].str.split(pat=';', n=4, expand=True)
df_full.drop(['LotInfo'], axis=1, inplace=True)

# replace 'NA' in LotFrontage with correct NaN
df_full['LotFrontage'] = df_full['LotFrontage'].replace('NA', np.nan)

# split exterior info and drop
df_full[['Exterior1st', 'ExterQual', 'ExterCond']] = df_full['Exterior'].str.split(pat=';', n=3, expand=True)
df_full.drop(['Exterior'], axis=1, inplace=True)

In [6]:
# detect all missing values
df_full.isna().sum().sort_values(ascending=False)

KitchenAbvGr    146
BsmtUnfSF        89
SalePrice        67
BsmtCond         44
LotFrontage      34
GarageType       34
BsmtFinType1     12
BsmtQual         12
LotArea           0
LotShape          0
Exterior1st       0
LotConfig         0
TotRmsAbvGrd      0
neighborhood      0
GrLivArea         0
OverallCond       0
BldgType          0
Electrical        0
Foundation        0
ExterQual         0
YearBuilt         0
uniqueID          0
OverallQual       0
WoodDeckSF        0
SaleType          0
CentralAir        0
Fireplaces        0
YrSold            0
HouseStyle        0
HeatingQC         0
RoofMatl          0
PavedDrive        0
FullBath          0
BedroomAbvGr      0
OpenPorchSF       0
RoofStyle         0
Utilities         0
BsmtFinSF1        0
Heating           0
KitchenQual       0
HalfBath          0
ExterCond         0
dtype: int64

In [7]:
# transform data type
for k, v in config['dtype_dict'].items():
	df_full[k] = df_full[k].astype(v)

## Missing Data

In [8]:
# missing number of kitchen is default to 1
df_full['KitchenAbvGr'] = df_full['KitchenAbvGr'].fillna(1)

# missing Salesprice from test file is default to 0
df_full['SalePrice'] = df_full['SalePrice'].fillna(0)

# impute BsmtUnfSF with median from neighborhood
df_full['BsmtUnfSF'] = df_full['BsmtUnfSF'].fillna(df_full['BsmtUnfSF'].median())

# missing BsmtCond is default to NA - no basement
df_full['BsmtCond'] = df_full['BsmtCond'].cat.add_categories('NA').fillna('NA')

# impute LotFrontage with median from neighborhood
df_full['LotFrontage'] = df_full['LotFrontage'].fillna(
	df_full.groupby('neighborhood')['LotFrontage'].transform('median'))

# missing GarageType is default to NA - no garage
df_full['GarageType'] = df_full['GarageType'].cat.add_categories('NA').fillna('NA')

# missing BsmtQual is default to NA - no basement
df_full['BsmtQual'] = df_full['BsmtQual'].cat.add_categories('NA').fillna('NA')

# missing BsmtFinType1 is default to NA - no basement
df_full['BsmtFinType1'] = df_full['BsmtFinType1'].cat.add_categories('NA').fillna('NA')

In [9]:
# remove duplicates
# df_full[df_full.duplicated(keep=False)]
df_full = df_full.drop_duplicates()

# remove house with Year sold less than Year Built
df_full = df_full[~(df_full.YrSold < df_full.YearBuilt)]

## Save data

In [10]:
# save full data set
df_full.to_csv(os.path.join(config['path']['train'], 'df_full_clean.csv'), index=False)